In [11]:
#lets start with importing some libraries
import pygame
# we need it for showing the maze and agent visually
import numpy as np
# we need it for math and matrix like q-table
import random
# we need it for generating random numbers
from collections import deque
# we need it for storing past positions ( I will do that to avoid loops and repetitive movements)
# It is time to set maze and learning items:
maze_size = 10
# we need maze with size of 10*10
start = (0, 0)
# start point for agent is 0,0
goal = (9,9)
num_obstacles = 15
#I think 15 obstacle is enough
#Let's define Q-learning parameters
alpha = 0.1
# alpha is learning rate
gamma = 0.95
#it is discount factor
epsilon = 1.0
#this is exploration rate
epsilon_min = 0.01
#smallest possible exploration
epsilon_decay = 0.999
episodes = 5000
#this is number of training rounds
max_steps = 100
#max steps per episode
#let's define the actions
actions = {
    0: (-1, 0), #for up
    1: (1, 0), #for down
    2: (0, -1), # for left
    3: (0, 1) #for right
}
# let's define Q-table initialization:
#Following this, this table stores the learned values for each cell and action
q_table = np.zeros((maze_size, maze_size, 4))
#Based on my professor the obstacles supposed to be different in each episod
#let's define the definition for creation of that:
def generate_obstacles():
    obstacles = set()
    while len(obstacles) < num_obstacles:
        x, y = random.randint(0, maze_size-1), random.randint(0, maze_size-1)
        if (x,y) !=start and (x, y) !=goal:
            obstacles.add((x, y))
    return obstacles
#we have to check if a position is valid or not. I mean that inside grid and not blocked.
def is_valid(pos, obstacles):
    x, y = pos
    return 0 <= x < maze_size and 0 <= y < maze_size and pos not in obstacles
# We have to check the maze is solvable or not, so
#I use BFS to confirm there is a path from strat to goal
#Let's define the definition:
def path_exists(start, goal, obstacles):
    queue = deque([start])
    visited = {start}
    while queue:
        x, y = queue.popleft()
        if (x, y) == goal:
            return True
        for dx, dy in actions.values():
            nx, ny = x + dx, y + dy
            if is_valid( (nx, ny), obstacles) and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append((nx, ny))
    return False
# There are many possibilities whenever agent wanted to move in the environment
# so, we have to change and predict his moves to reach the better and faster result
def step(pos, action, obstacles):
    dx, dy = actions[action]
    new_pos = (pos[0] + dx, pos[1] + dy)
    if not is_valid(new_pos, obstacles):
        return pos, -1 # this would be for hiting the wall or obstacles and causes penalty
    if new_pos == goal:
        return new_pos,100 # if agent will reach the goal he will receive big reward
    return new_pos, -0.5  #I set the small penalty for his regular step
# let's train the agent using the Q-learning
for ep in range(episodes):
    obs=generate_obstacles()
    while not path_exists(start, goal, obs): # for make sure about maze is solvable
        obs=generate_obstacles()
    pos = start
    for _ in range(max_steps):
#Decide whether to explore or use best known move
        if np.random.rand() < epsilon:
            action = random.choice(list(actions.keys()))
        else:
            action = np.argmax(q_table[pos[0], pos[1]])
        next_pos, reward = step(pos,action, obs)
        next_max = np.max(q_table[next_pos[0], next_pos[1]])
#so, let's update Q-table using the Q-learning formula
        q_table[pos[0], pos[1], action] += alpha * (
            reward +gamma * next_max - q_table[pos[0], pos[1], action])
        pos = next_pos
        if pos == goal:
            break
#another important thing would be slowly reduce exploration over time

    epsilon = max(epsilon_min, epsilon * epsilon_decay)
#As we have learned during the class, we have to define pygame setup for visual display:
pygame.init()
cell_size = 600 // maze_size #this is the size of each cell on the screen
#so let's set the windows size
screen = pygame.display.set_mode((600, 600))
pygame.display.set_caption("smart agent (no loops)")
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 36)
#Let's draw maze, obstacles, agent, and goal:
def draw(maze, agent, message=""):
#I want to have ligh green background because of the real maze game appearance,so:
    screen.fill((230, 255, 230))
    for x in range(maze_size):
        for y in range(maze_size):
            rect = pygame.Rect(y * cell_size, x * cell_size, cell_size, cell_size)
            if (x, y) in maze:
                pygame.draw.rect(screen, (0, 100, 0), rect) #obstacles are supposed to be dark green
            elif (x, y) ==goal:
                pygame.draw.rect(screen, (255, 0, 0), rect) #goal color is supposed to be red
            elif (x, y) == agent:
                pygame.draw.rect(screen, (0, 0, 255), rect) #For the color of agent and it would be blue
            else:
                pygame.draw.rect(screen, (200, 255, 200), rect, 1) #For grid lines
    if message:
        text = font.render(message, True, (0, 128, 0))
        screen.blit(text, (10, 10))
    pygame.display.flip()
# I have to limit agent's movements whenever it runs through Maze
def run_agent(obstacles):
    pos = start
    prev_positions = deque(maxlen=6) #For storing recent positions to detect cycles
    for _ in range(max_steps):
        draw(obstacles, pos)
        clock.tick(5)
#choosing best action from Q-table
        q_values = q_table[pos[0], pos[1]]
        action_order = list(np.argsort(q_values)[::-1])

        #For trying the best actions first to have a better performance.
        moved = False
        for a in action_order:
            dx, dy = actions[a]
            new_pos = (pos[0] + dx, pos[1] + dy)
            if not is_valid(new_pos, obstacles):
                continue
            if new_pos in prev_positions:
                continue #For avoiding the loops
            prev_positions.append(pos)
            pos = new_pos
            moved = True
            break
        #For handling the window close event we have
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); exit()
        #For time that agent is stuck or looping
        if not moved:
            draw(obstacles, pos, "Agent stuck or looping")
            pygame.time.wait(1500)
            return False
        #For the time that agent reached the goal
        if pos == goal:
            draw(obstacles, pos, "Agent reached the goal!!")
            pygame.time.wait(1500)
            return True
    return False
#For showing 5 successful runs visually
successes = 0
while successes < 5:
    obs = generate_obstacles()
    if path_exists(start, goal, obs):
        if run_agent(obs):
            successes += 1
pygame.quit()
